In [138]:
import os
import os
import numpy as np
import json
import argparse
import h5py
import numpy as np
import torch
import scipy.stats as ss
from tqdm import tqdm_notebook as tqdm


In [139]:
%cd decoding

[Errno 2] No such file or directory: 'decoding'
/Users/gilad/Desktop/Projects/semantic-decoding/decoding


In [84]:
REPO_DIR = os.getcwd()
DATA_LM_DIR = os.path.join(REPO_DIR, "data_lm")
DATA_TRAIN_DIR = os.path.join(REPO_DIR, "data_train")
DATA_TEST_DIR = os.path.join(REPO_DIR, "data_test")
MODEL_DIR = os.path.join(REPO_DIR, "models")
RESULT_DIR = os.path.join(REPO_DIR, "results")
SCORE_DIR = os.path.join(REPO_DIR, "scores")

# GPT encoding model parameters

TRIM = 5
STIM_DELAYS = [1, 2, 3, 4]
RESP_DELAYS = [-4, -3, -2, -1]
ALPHAS = np.logspace(1, 3, 10)
NBOOTS = 50
VOXELS = 10000
CHUNKLEN = 40
GPT_LAYER = 9
GPT_WORDS = 5

# decoder parameters

RANKED = True
WIDTH = 200
NM_ALPHA = 2/3
LM_TIME = 8
LM_MASS = 0.9
LM_RATIO = 0.1
EXTENSIONS = 5

# evaluation parameters

WINDOW = 20

# devices

GPT_DEVICE = "cpu"
EM_DEVICE = "cpu"
SM_DEVICE = "cpu"

In [86]:
#import config
from GPT import GPT


## Decoding Parameters

In [87]:
subject = "UTS01"
experiment = "perceived_speech"
task = "ifthishaircouldtalk"

In [88]:
REPO_DIR = os.path.dirname(os.getcwd())

In [89]:
DATA_LM_DIR = os.path.join(REPO_DIR, "data_lm")
DATA_TRAIN_DIR = os.path.join(REPO_DIR, "data_train")
DATA_TEST_DIR = os.path.join(REPO_DIR, "data_test")
MODEL_DIR = os.path.join(REPO_DIR, "models")
RESULT_DIR = os.path.join(REPO_DIR, "results")
SCORE_DIR = os.path.join(REPO_DIR, "scores")

In [90]:
gpt_checkpoint = "perceived"

In [91]:
word_rate_voxels = "auditory"

## Load fMRI response

In [92]:
hf = h5py.File(os.path.join(DATA_TEST_DIR, "test_response", subject, experiment, task + ".hf5"), "r")
resp = np.nan_to_num(hf["data"][:])
hf.close()

In [93]:
resp.shape

(249, 81126)

## Load GPT 

In [94]:
INIT = ['i', 'we', 'she', 'he', 'they', 'it']
STOPWORDS = {'is', 'does', 's', 'having', 'doing', 'these', 'shan', 'yourself', 'other', 'are', 'hasn', 'at', 'for', 'while', 'down', "hadn't", 'until', 'above', 'during', 'each', 'now', 'have', "won't", 'once', 'why', 'here', 'ourselves', 'to', 'over', 'into', 'who', 'that', 'myself', 'he', 'themselves', 'were', 'against', 'about', 'some', 'has', 'but', 'ma', 'their', 'this', 'there', 'with', "that'll", "shan't", "wouldn't", 'a', 'those', "you'll", 'll', 'few', 'couldn', 'an', 'd', "weren't", 'doesn', 'own', 'won', 'didn', 'what', 'when', 'in', 'below', 'where', "it's", 'most', 'just', "you're", 'yourselves', 'too', "don't", "she's", "didn't", "hasn't", 'isn', "mustn't", 'of', 'did', 'how', 'himself', 'aren', 'if', 'very', 'or', 'weren', 'it', 'be', 'itself', "doesn't", 'my', 'o', 'no', "isn't", 'before', 'after', 'off', 'was', 'can', 'the', 'been', 'her', 'him', "wasn't", 've', 'through', "needn't", 'because', 'nor', 'will', 'm', 't', 'out', 'on', 'she', 'all', 'then', 'than', "mightn't", 'hers', 'herself', 'only', 'should', 're', 'ain', 'wasn', "aren't", "couldn't", 'they', 'hadn', 'had', 'more', 'and', 'under', "shouldn't", 'any', 'y', 'don', 'from', 'so', 'whom', 'as', 'mustn', 'between', 'up', 'do', 'both', 'such', 'our', 'its', 'which', 'not', "haven't", 'needn', 'by', "should've", 'again', 'shouldn', 'his', 'me', 'further', 'yours', 'am', 'your', 'haven', 'wouldn', 'being', 'ours', 'you', 'i', 'theirs', 'mightn', 'same', 'we', "you've", 'them', "you'd"}

In [95]:
GPT_LAYER = 9
GPT_WORDS = 5

In [96]:
GPT_DEVICE = "cpu"

In [97]:
os.path.join(DATA_LM_DIR, "decoder_vocab.json")

'/Users/gilad/Desktop/Projects/semantic-decoding/data_lm/decoder_vocab.json'

In [98]:
os.path.join(DATA_LM_DIR, gpt_checkpoint, "vocab.json")

'/Users/gilad/Desktop/Projects/semantic-decoding/data_lm/perceived/vocab.json'

In [99]:
with open(os.path.join(DATA_LM_DIR, gpt_checkpoint, "vocab.json"), "r") as f:
    gpt_vocab = json.load(f)


In [100]:
with open(os.path.join(DATA_LM_DIR, "decoder_vocab.json"), "r") as f:
    decoder_vocab = json.load(f)


In [101]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")


In [102]:
class LMFeatures():
    """class for extracting contextualized features of stimulus words
    """
    def __init__(self, model, layer, context_words):
        self.model, self.layer, self.context_words = model, layer, context_words

    def extend(self, extensions, verbose = False):
        """outputs array of vectors corresponding to the last words of each extension
        """
        contexts = [extension[-(self.context_words+1):] for extension in extensions]
        if verbose: print(contexts)
        context_array = self.model.get_context_array(contexts)
        embs = self.model.get_hidden(context_array, layer = self.layer)
        return embs[:, len(contexts[0]) - 1]

    def make_stim(self, words):
        """outputs matrix of features corresponding to the stimulus words
        """
        context_array = self.model.get_story_array(words, self.context_words)
        embs = self.model.get_hidden(context_array, layer = self.layer)
        return np.vstack([embs[0, :self.context_words], 
            embs[:context_array.shape[0] - self.context_words, self.context_words]])

In [103]:
def get_nucleus(probs, nuc_mass, nuc_ratio):
    """identify words that constitute a given fraction of the probability mass
    """
    nuc_ids = np.where(probs >= np.max(probs) * nuc_ratio)[0]
    nuc_pairs = sorted(zip(nuc_ids, probs[nuc_ids]), key = lambda x : -x[1]) 
    sum_mass = np.cumsum([x[1] for x in nuc_pairs])
    cutoffs = np.where(sum_mass >= nuc_mass)[0]
    if len(cutoffs) > 0: nuc_pairs = nuc_pairs[:cutoffs[0]+1]
    nuc_ids = [x[0] for x in nuc_pairs]                     
    return nuc_ids


In [104]:
def context_filter(proposals, context):
    """filter out words that occur in a context to prevent repetitions
    """
    cut_words = []
    cut_words.extend([context[i+1] for i, word in enumerate(context[:-1]) if word == context[-1]]) # bigrams
    cut_words.extend([x for x in proposals if x not in STOPWORDS and in_context(x, context)]) # unigrams
    return [x for x in proposals if x not in cut_words]


In [105]:
def in_context(word, context):
    """test whether [word] or a stem of [word] is in [context]
    """
    stem_context = [stemmer.stem(x) for x in context]
    stem_word = stemmer.stem(word)
    return (stem_word in stem_context or stem_word in context)


In [106]:
class LanguageModel():
    """class for generating word sequences using a language model
    """
    def __init__(self, model, vocab, nuc_mass = 1.0, nuc_ratio = 0.0):        
        self.model = model
        self.ids = {i for word, i in self.model.word2id.items() if word in set(vocab)}
        self.nuc_mass, self.nuc_ratio = nuc_mass, nuc_ratio
        
    def ps(self, contexts):
        """get probability distributions over the next words for each context
        """
        context_arr = self.model.get_context_array(contexts)
        probs = self.model.get_probs(context_arr)
        return probs[:, len(contexts[0]) - 1] 
    
    def beam_propose(self, beam, context_words):
        """get possible extension words for each hypothesis in the decoder beam
        """
        if len(beam) == 1: 
            nuc_words = [w for w in INIT if self.model.word2id[w] in self.ids]
            nuc_logprobs = np.log(np.ones(len(nuc_words)) / len(nuc_words))
            return [(nuc_words, nuc_logprobs)]
        else:
            contexts = [hyp.words[-context_words:] for hyp in beam]
            beam_probs = self.ps(contexts)
            beam_nucs = []
            for context, probs in zip(contexts, beam_probs):
                nuc_ids = get_nucleus(probs, nuc_mass = self.nuc_mass, nuc_ratio = self.nuc_ratio)
                nuc_words = [self.model.vocab[i] for i in nuc_ids if i in self.ids]
                nuc_words = context_filter(nuc_words, context)
                nuc_logprobs = np.log([probs[self.model.word2id[w]] for w in nuc_words])
                beam_nucs.append((nuc_words, nuc_logprobs))
            return beam_nucs

In [107]:
gpt = GPT(path = os.path.join(DATA_LM_DIR, gpt_checkpoint, "model"), vocab = gpt_vocab, device = GPT_DEVICE)
features = LMFeatures(model = gpt, layer = GPT_LAYER, context_words = GPT_WORDS)
lm = LanguageModel(gpt, decoder_vocab, nuc_mass = LM_MASS, nuc_ratio = LM_RATIO)


## Load Encoding Model

In [108]:
%pwd

'/Users/gilad/Desktop/Projects/semantic-decoding/decoding'

In [109]:
models_dir = "/Users/gilad/Desktop/Projects/semantic-decoding/decoding/models/UTS01"

In [110]:
model_file = "encoding_model_percieved.npz"

In [111]:
load_location = os.path.join(MODEL_DIR, subject)
word_rate_model = np.load(os.path.join(load_location, "word_rate_model_%s.npz" % word_rate_voxels), allow_pickle = True)
#encoding_model = np.load(os.path.join(load_location, "encoding_model_%s.npz" % gpt_checkpoint))
encoding_model = np.load(models_dir + "/" + model_file)
weights = encoding_model["weights"]
noise_model = encoding_model["noise_model"]
tr_stats = encoding_model["tr_stats"]
word_stats = encoding_model["word_stats"]

In [112]:
def make_delayed(stim, delays, circpad=False):
    """Creates non-interpolated concatenated delayed versions of [stim] with the given [delays] 
    (in samples).
    
    If [circpad], instead of being padded with zeros, [stim] will be circularly shifted.
    """
    nt,ndim = stim.shape
    dstims = []
    for di,d in enumerate(delays):
        dstim = np.zeros((nt, ndim))
        if d<0: ## negative delay
            dstim[:d,:] = stim[-d:,:]
            if circpad:
                dstim[d:,:] = stim[:-d,:]
        elif d>0:
            dstim[d:,:] = stim[:-d,:]
            if circpad:
                dstim[:d,:] = stim[-d:,:]
        else: ## d==0
            dstim = stim.copy()
        dstims.append(dstim)
    return np.hstack(dstims)

In [113]:
def predict_word_rate(resp, wt, vox, mean_rate):
    """predict word rate at each acquisition time
    """
    delresp = make_delayed(resp[:, vox], RESP_DELAYS)
    rate = ((delresp.dot(wt) + mean_rate)).reshape(-1).clip(min = 0)
    return np.round(rate).astype(int)

def predict_word_times(word_rate, resp, starttime = 0, tr = 2):
    """predict evenly spaced word times from word rate
    """
    half = tr / 2
    trf = TRFile(None, tr)
    trf.soundstarttime = starttime
    trf.simulate(resp.shape[0])
    tr_times = trf.get_reltriggertimes() + half

    word_times = []
    for mid, num in zip(tr_times, word_rate):  
        if num < 1: continue
        word_times.extend(np.linspace(mid - half, mid + half, num, endpoint = False) + half / num)
    return np.array(word_times), tr_times

In [114]:
EM_DEVICE = "cpu"

In [115]:
class EncodingModel():
    """class for computing the likelihood of observing brain recordings given a word sequence
    """
    def __init__(self, resp, weights, voxels, sigma, device = "cpu"):
        self.device = device
        self.weights = torch.from_numpy(weights[:, voxels]).float().to(self.device)
        self.resp = torch.from_numpy(resp[:, voxels]).float().to(self.device)
        self.sigma = sigma
        
    def set_shrinkage(self, alpha):
        """compute precision from empirical covariance with shrinkage factor alpha
        """
        precision = np.linalg.inv(self.sigma * (1 - alpha) + np.eye(len(self.sigma)) * alpha)
        self.precision = torch.from_numpy(precision).float().to(self.device)

    def prs(self, stim, trs):
        """compute P(R | S) on affected TRs for each hypothesis
        """
        with torch.no_grad(): 
            stim = stim.float().to(self.device)
            diff = torch.matmul(stim, self.weights) - self.resp[trs] # encoding model residuals
            multi = torch.matmul(torch.matmul(diff, self.precision), diff.permute(0, 2, 1))
            return -0.5 * multi.diagonal(dim1 = -2, dim2 = -1).sum(dim = 1).detach().cpu().numpy()

In [116]:
em = EncodingModel(resp, weights, encoding_model["voxels"], noise_model, device = EM_DEVICE)

In [117]:
NM_ALPHA =  2/3

In [118]:
em.set_shrinkage(NM_ALPHA)

In [119]:
class TRFile(object):
    def __init__(self, trfilename, expectedtr=2.0045):
        """Loads data from [trfilename], should be output from stimulus presentation code.
        """
        self.trtimes = []
        self.soundstarttime = -1
        self.soundstoptime = -1
        self.otherlabels = []
        self.expectedtr = expectedtr
        
        if trfilename is not None:
            self.load_from_file(trfilename)
        

    def load_from_file(self, trfilename):
        """Loads TR data from report with given [trfilename].
        """
        ## Read the report file and populate the datastructure
        for ll in open(trfilename):
            timestr = ll.split()[0]
            label = " ".join(ll.split()[1:])
            time = float(timestr)

            if label in ("init-trigger", "trigger"):
                self.trtimes.append(time)

            elif label=="sound-start":
                self.soundstarttime = time

            elif label=="sound-stop":
                self.soundstoptime = time

            else:
                self.otherlabels.append((time, label))
        
        ## Fix weird TR times
        itrtimes = np.diff(self.trtimes)
        badtrtimes = np.nonzero(itrtimes>(itrtimes.mean()*1.5))[0]
        newtrs = []
        for btr in badtrtimes:
            ## Insert new TR where it was missing..
            newtrtime = self.trtimes[btr]+self.expectedtr
            newtrs.append((newtrtime,btr))

        for ntr,btr in newtrs:
            self.trtimes.insert(btr+1, ntr)

    def simulate(self, ntrs):
        """Simulates [ntrs] TRs that occur at the expected TR.
        """
        self.trtimes = list(np.arange(ntrs)*self.expectedtr)
    
    def get_reltriggertimes(self):
        """Returns the times of all trigger events relative to the sound.
        """
        return np.array(self.trtimes)-self.soundstarttime

    @property
    def avgtr(self):
        """Returns the average TR for this run.
        """
        return np.diff(self.trtimes).mean()

In [120]:
def lanczosfun(cutoff, t, window=3):
    """Compute the lanczos function with some cutoff frequency [B] at some time [t].
    [t] can be a scalar or any shaped numpy array.
    If given a [window], only the lowest-order [window] lobes of the sinc function
    will be non-zero.
    """
    t = t * cutoff
    val = window * np.sin(np.pi*t) * np.sin(np.pi*t/window) / (np.pi**2 * t**2)
    val[t==0] = 1.0
    val[np.abs(t)>window] = 0.0
    return val# / (val.sum() + 1e-10)


In [121]:
def get_lanczos_mat(oldtime, newtime, window = 3, cutoff_mult = 1.0, rectify = False):
    """get matrix for downsampling from TR times to word times
    """
    cutoff = 1 / np.mean(np.diff(newtime)) * cutoff_mult
    sincmat = np.zeros((len(newtime), len(oldtime)))
    for ndi in range(len(newtime)):
        sincmat[ndi,:] = lanczosfun(cutoff, newtime[ndi] - oldtime, window)
    return sincmat


## predict word times

In [122]:
word_rate = predict_word_rate(resp, word_rate_model["weights"], word_rate_model["voxels"], word_rate_model["mean_rate"])


In [123]:
word_times, tr_times = predict_word_times(word_rate, resp, starttime = -10)


In [124]:
lanczos_mat = get_lanczos_mat(word_times, tr_times)


/var/folders/mr/9jcsmzqd1312jvpkw6kf0y0c0000gn/T/ipykernel_82114/4071827947.py:8: RuntimeWarning: invalid value encountered in divide
  val = window * np.sin(np.pi*t) * np.sin(np.pi*t/window) / (np.pi**2 * t**2)


## decode responses

In [125]:
class Hypothesis(object):
    """a class for representing word sequence hypotheses
    """
    def __init__(self, parent = None, extension = None):
        if parent is None: 
            self.words, self.logprobs, self.embs = [], [], []
        else:
            word, logprob, emb = extension
            self.words = parent.words + [word]
            self.logprobs = parent.logprobs + [logprob]
            self.embs = parent.embs + [emb]

In [126]:
class Decoder(object):
    """class for beam search decoding
    """
    def __init__(self, word_times, beam_width, extensions = 5):
        self.word_times = word_times
        self.beam_width, self.extensions = beam_width, extensions
        self.beam = [Hypothesis()] # initialize with empty hypothesis
        self.scored_extensions = [] # global extension pool
        
    def first_difference(self):
        """get first index where hypotheses on the beam differ
        """
        words_arr = np.array([hypothesis.words for hypothesis in self.beam])
        if words_arr.shape[0] == 1: return words_arr.shape[1]
        for index in range(words_arr.shape[1]): 
            if len(set(words_arr[:, index])) > 1: return index
        return 0
    
    def time_window(self, sample_index, seconds, floor = 0):
        """number of prior words within [seconds] of the currently sampled time point"""
        window = [time for time in self.word_times if time < self.word_times[sample_index] 
                  and time > self.word_times[sample_index] - seconds]
        return max(len(window), floor)
        
    def get_hypotheses(self):
        """get the number of permitted extensions for each hypothesis on the beam
        """
#        print("get hypotheses")
        if len(self.beam[0].words) == 0: 
            return zip(self.beam, [self.extensions for hypothesis in self.beam])
        logprobs = [sum(hypothesis.logprobs) for hypothesis in self.beam]
        num_extensions = [int(np.ceil(self.extensions * rank / len(logprobs))) for 
                          rank in ss.rankdata(logprobs)]
        
        return zip(self.beam, num_extensions)
    
    def add_extensions(self, extensions, likelihoods, num_extensions):
        """add extensions for each hypothesis to global extension pool
        """
        scored_extensions = sorted(zip(extensions, likelihoods), key = lambda x : -x[1])
        self.scored_extensions.extend(scored_extensions[:num_extensions])

    def extend(self, verbose = False):
        """update beam based on global extension pool 
        """
        self.beam = [x[0] for x in sorted(self.scored_extensions, key = lambda x : -x[1])[:self.beam_width]]
        self.scored_extensions = []
        if verbose: print(self.beam[0].words)
        
    def save(self, path):
        """save decoder results
        """
        np.savez(path, words = np.array(self.beam[0].words), times = np.array(self.word_times))
    

In [127]:
def affected_trs(start_index, end_index, lanczos_mat, delay = True):
    """identify TRs influenced by words in the range [start_index, end_index]
    """
    start_tr, end_tr = np.where(lanczos_mat[:, start_index])[0][0], np.where(lanczos_mat[:, end_index])[0][-1]
    start_tr, end_tr = start_tr + min(STIM_DELAYS), end_tr + max(STIM_DELAYS)
    start_tr, end_tr = max(start_tr, 0), min(end_tr, lanczos_mat.shape[0] - 1)
    return np.arange(start_tr, end_tr + 1)


In [128]:
class StimulusModel():
    """class for constructing stimulus features
    """
    def __init__(self, lanczos_mat, tr_stats, word_mean, device = 'cpu'):
        self.device = device
        self.lanczos_mat = torch.from_numpy(lanczos_mat).float().to(self.device)
        self.tr_mean = torch.from_numpy(tr_stats[0]).float().to(device)
        self.tr_std_inv = torch.from_numpy(np.diag(1 / tr_stats[1])).float().to(device)
        self.blank = torch.from_numpy(word_mean).float().to(self.device)
        
    def _downsample(self, variants):
        """downsamples word embeddings to TR embeddings for each hypothesis
        """
        return torch.matmul(self.lanczos_mat.unsqueeze(0), variants)
    
    def _normalize(self, tr_variants):
        """normalize TR embeddings for each hypothesis
        """
        centered = tr_variants - self.tr_mean
        return torch.matmul(centered, self.tr_std_inv)

    def _delay(self, tr_variants, n_vars, n_feats):
        """apply finite impulse response delays to TR embeddings
        """
        delays = STIM_DELAYS
        n_trs = tr_variants.shape[1]
        del_tr_variants = torch.zeros(n_vars, n_trs, len(delays)*n_feats).to(self.device)
        for c, d in enumerate(delays): 
            feat_ind_start = c * n_feats
            feat_ind_end = (c + 1) * n_feats
            del_tr_variants[:, d:, feat_ind_start:feat_ind_end] = tr_variants[:, :n_trs - d, :]
        return del_tr_variants
        
    def make_variants(self, sample_index, hypothesis_embs, var_embs, affected_trs):
        """create stimulus features for each hypothesis
        """
        n_variants, n_feats = len(var_embs), self.blank.shape[0]
        with torch.no_grad():
            full = self.blank.repeat(self.lanczos_mat.shape[1], 1) # word times x features
            full[:sample_index] = torch.tensor(np.array(hypothesis_embs)).float().reshape(-1, n_feats).to(self.device)
            variants = full.repeat(n_variants, 1, 1) # variants x word times x features
            variants[:, sample_index, :] = torch.tensor(np.array(var_embs)).float().to(self.device)
            tr_variants = self._normalize(self._downsample(variants))
            del_tr_variants = self._delay(tr_variants, n_variants, n_feats)
        return del_tr_variants[:, affected_trs, :].to('cpu')

In [134]:
decoder = Decoder(word_times, WIDTH)

In [135]:
sm = StimulusModel(lanczos_mat, tr_stats, word_stats[0], device = SM_DEVICE)

In [136]:
decoder.get_hypotheses()

In [137]:
for sample_index in tqdm(range(len(word_times))):
    trs = affected_trs(decoder.first_difference(), sample_index, lanczos_mat)
    ncontext = decoder.time_window(sample_index, LM_TIME, floor = 5)
    beam_nucs = lm.beam_propose(decoder.beam, ncontext)
#    print(beam_nucs)
    for c, (hyp, nextensions) in enumerate(decoder.get_hypotheses()):
#        print(c)
        nuc, logprobs = beam_nucs[c]
#        print(nuc)
#        print("hyp.embs: ", hyp.embs)
        if len(nuc) < 1: continue
        extend_words = [hyp.words + [x] for x in nuc]
        extend_embs = list(features.extend(extend_words))
#         print("sample_index", sample_index)
#         print("len(hyp.embs)", len(hyp.embs))
#         print("len(extend_embs)", len(extend_embs) )
#         print("trs", trs)
        stim = sm.make_variants(sample_index, hyp.embs, extend_embs, trs)
        
        likelihoods = em.prs(stim, trs)
#        print("liklihoods: ", likelihoods)
        local_extensions = [Hypothesis(parent = hyp, extension = x) for x in zip(nuc, logprobs, extend_embs)]
        decoder.add_extensions(local_extensions, likelihoods, nextensions)
    decoder.extend(verbose = False)


/var/folders/mr/9jcsmzqd1312jvpkw6kf0y0c0000gn/T/ipykernel_82114/989296795.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sample_index in tqdm(range(len(word_times))):


  0%|          | 0/1360 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [133]:
" ".join(decoder.beam[0].words)

'we need to get your picture taken for id'

In [ ]:
 I'm from a very small African country called Zimbabwe. It's a country that's been in the news very recently for very many bad reasons. The one thing you might not know about Zimbabwe is that it's one of the youngest democracies in the world. Well, democracy is perhaps not the right word, so let me say it's one of the youngest countries in the world. It's only 29 years old. And the big year of change in my country, the big year of change in my family, and the big year of change for me was 1980, when my country finally became independent. Those of you, and I'm sure there are many of you here who are experts in African history, will know that in the late 50s and the early 60s, Britain, France, and the other colonial powers were giving up their colonies for a number of reasons. So countries like Nigeria became independent, Ghana became independent, but not Rhodesia. The white minority government in Rhodesia, led by somebody called Ian Smith, had other ideas for the kind of country they wanted to live in. And one of the very firm ideas they had was that they didn't want to live in a country ruled by black people. So they declared a unilateral declaration of independence from Britain, the colonial power, and the result of this was that the country was isolated from the outside. And on the inside, there was a civil war between the white minority on the one side and the black majority led by black freedom fighters. And after about 14 years of war and negotiations in England, we finally became an independent republic called Zimbabwe. I have very vivid memories of that time. We lived in the township, which were these African areas. Rhodesia was segregated along the same lines as South Africa, but on a smaller scale. So we lived in the township, where I remember around the time of independence, there was so much music. Everybody was singing, everybody was dancing, it was almost like you could actually touch the joy in the air. And the song that everybody was singing, if you'll allow me to sing it, is a song by Bob Marley called Zimbabwe. Do you know it? Then join in. Africa shall liberate Zimbabwe, Africa shall liberate Zimbabwe. It was a song that was on everybody's lips. Then the biggest thing that happened that year for a lot of people in Harare was that Bob Marley himself came to Salisbury to give a concert, an independence concert. He came all the way from Kingston with his whalers. But that was not the biggest thing that happened to me, because the biggest thing that happened to me was that the white areas, the formerly white areas, the suburbs, began to open up. And my father finally achieved the dream of a lifetime. He moved us out of the township into the suburbs to live with the white people, the good area. So imagine what it meant for a family from the township, where the only road that was tarred was the road to town. And all the other little roads were dusty, full of mud, full of dust. There was no electricity at night in some parts of the township. So imagine us in this new environment where the roads are lined with beautiful trees. In the morning, the milkman deposits two bottles of milk outside your door, one silver, one gold, depending on the amount of cream you want in your milk. The breadman rings the bell in the morning to tell you that your fresh Lobos bread is ready for you outside. And the newspaper boy throws his newspaper over the wall for you to read in the morning. I went to a school called Alfred Bight, where I found myself as one of 24 children in the classroom. Twenty of them were white. Now, I had been at a school in the township where we had something called hot seating, which meant that 48 of us came to school in the morning, and then went home to make room for another 48 people who came in the afternoon. So this was absolute paradise to me, a class of only 24 children. But the very first thing I did in my new classroom was the wrong thing. My teacher, Miss Callan, called me to her desk, and as a well-trained little African girl who had been brought up well by her teachers, I knelt before her. What are you, a goat? she said. I still remember the surprised laughter of the whole class, and it was a sound that I became very familiar with as the year went on, because it seemed everything about me was wrong, everything I did was wrong. My hair, for instance, not this hair. Well, this is my hair in the sense that I paid for it. But my hair was too curly, it was too close to my head. My English, when it came, was too slow, and the accent was very strange. And then there was the small matter of the sandwiches. You see, my mother made us egg sandwiches every morning for school. So she fried eggs, put them between butter slices of bread. That was the wrong food to take to school, because what the white children ate was something called polony, which really stank. And then they had something else called marmite, which is a yeast extract, and it's the foulest tasting substance known to man. And I just, I really wanted this stinky, horrible food, because I thought that, you know, if I ate the same food that the other people ate, if I had the same kind of hair, then I would fit in somehow. But of course, that didn't happen. So every day I had Russell Webb laughing at my hair, I had Carrie Trelaw laughing at my hair, I had Natasha Russell refusing to share her Smarties with me that she bought on holiday in South Africa. The only time that I really felt I belonged to Alfred Bight, my new school, was in the mornings at assembly. We would sit cross-legged on the floor, and Miss Roberts, our headmistress, would play the piano and lead us in the school song. It was a song about valor, about duty, about honor. It was a song about commitment and dedication. And I sang it at the top of my voice because it was the one moment when I knew peace at my school, when there was no laughter, when there was no mockery on the playground. It was a song about the pioneer column who colonized my country and turned it into Rhodesia. So there I was, a 10-year-old African child in a newly independent country called Zimbabwe, singing this song in which God regarded the conquest of my country as an act of honor. This was a song celebrating the conquest of a kingdom in Zimbabwe. It was a song celebrating the fact that many thousands of people had been made landless. The song was called, Thou Who Didst Guide Our Father's Feet. And the last sentence was, As thou hast done, do once again. I loved that song. As you can imagine, we didn't sing it for very long. My friend, Jessie Majome, who has gone on to greater things and is now actually the Deputy Minister of Justice in my country, told her father about the song that we were made to sing every morning. And her father called the Herald, which was the state newspaper. And I remember the Herald journalist coming to the school and Miss Roberts rushing across the quadrangle. And she was quite baffled by the whole thing. I'm not a racialist, she insisted. I have black children in my school. What do you mean I'm a racist? That's not it at all. It's just a tradition. It's like the school motto. It's like the recorder lessons. It's just a tradition. It doesn't mean anything. I think that this was the moment that the teachers at my school and Miss Roberts finally had to confront what it meant to live in an independent African country. It wasn't just about changing the name of the country from Rhodesia to Zimbabwe, changing the name of the capital from Sorse to Harare. It meant that not only did we have to start relating to each other differently across the racial divide, but we also had to start reevaluating our history. And for some of the teachers, I think that was a step too far. Because about a year later came the great white flight, when a lot of teachers left the school and a lot of children left the school as well. I'm not really sure that I can relate the white flight to this particular incident. But what I know is that after about a year at my new school, it finally began to resemble the kind of school you'd expect to find in an independent African country. So I finally found myself being part of a school that had all the amenities that my old township lacked, but that truly looked like a Zimbabwean school. And this is how independence came to me. Thank you.
